    from IPython import get_ipython; ip = get_ipython()
    %load_ext pidgin.post_run_cell
    
`import pidgin.post_run_cell` adds extra steps on `'post_run_cell' in ip.events.callbacks`. After the normal 
`ip.run_cell` evaluation:
    
1. `import doctest`s are executed.
2. Inline code statements and expressions are `exec`uted.
3. The raw source entered into `ip.run_cell` if it should be displayed.

    `import pidgin.display` controls the display behaviors and `import pidgin.tangle` controls
    how the __Markdown__ source is converted to __Python__.

In [1]:
    import IPython, functools, types, textwrap, ast, sys, traceback

1. `run_cell_doctests` _will discover any doctests in the cell and evaluate them_.
2. `run_user_expressions` _`exec`ute inline code statements and expressions to assure that it is valid code_.
3. `run_cell_display` _display the input_.

`show_error` decorates a `post_run_cell` `callable` to show an `Exception` in the result.

In [2]:
    def show_error(ip, object):
        @functools.wraps(object)
        def post_run_cell(result: "Return from `ip.run_cell`"):
            if result.error_in_exec or result.error_before_exec: return result
            result.error_in_exec = object(result)
            if result.error_in_exec:
                ip.showtraceback(exc_to_tuple(result.error_in_exec))
            return result
        return post_run_cell

`exc_to_tuple` converts an `Exception` to a `tuple` to display the traceback__.

In [3]:
    def exc_to_tuple(Exception): return type(Exception), Exception, Exception.__traceback__

# Extensions

In [4]:
    def create_load_extension(callable):
        @functools.wraps(callable)
        def load_ipython_extension(ip): 
            _show_error = types.MethodType(show_error, ip)
            ip.events.register('post_run_cell', _show_error(types.MethodType(callable, ip)))
        return load_ipython_extension

                
    def create_unload_extension(callable):
        @functools.wraps(callable)
        def unload_ipython_extension(ip): 
            ip.events.callbacks['post_run_cell'] = [
                object for object in ip.events.callbacks['post_run_cell']
                if object.__name__  != callable.__name__]
        return unload_ipython_extension
        

In [5]:
    if __name__ == '__main__': 
        %reload_ext pidgin.tangle
        %reload_ext pidgin.display
        with __import__('importnb').Notebook():
            try: 
                from .tangle import markdown_to_python
                from . import doctesting
            except: 
                from tangle import markdown_to_python
                import doctesting
        unload_ipython_extension = create_unload_extension(doctesting.run_cell_doctests)
        unload_ipython_extension(IPython.get_ipython())
        load_ipython_extension = create_load_extension(doctesting.run_cell_doctests)
        load_ipython_extension(IPython.get_ipython())